<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/CalcSynergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library 



In [1]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

real	1m3.731s
user	0m55.661s
sys	0m5.710s


In [ ]:
! pip install keras
! pip install tensorflow
! pip install keras-gcn
! pip install numpy
! pip install matplotlib
! pip install scipy
! pip install deepchem
! pip install keras-tuner
! pip install chembl_webresource_client

In [6]:
#import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers
import pandas as pd
import numpy as np
#import deepchem as dc
#from rdkit import Chem
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import BayesianOptimization
from chembl_webresource_client.new_client import new_client

#Boolean GRN

#Target Protein

## STITCH

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/kittisaktaoma/colab_practice/master/sample.txt",sep = "\t")
df_list_drug = pd.concat([df["drug_row"], df["drug_col"]]).unique().tolist()

In [10]:
df.head

<bound method NDFrame.head of        block_id                 drug_row  ... cell_line_name  cellosaurus_accession
0         16145                     5-FU  ...        EFM192B              CVCL_1813
1         16146                     5-FU  ...        EFM192B              CVCL_1813
2         16147                     5-FU  ...        EFM192B              CVCL_1813
3         16148                     5-FU  ...        EFM192B              CVCL_1813
4         16149                     5-FU  ...        EFM192B              CVCL_1813
...         ...                      ...  ...            ...                    ...
45610    443886               VISMODEGIB  ...     MDA-MB-468              CVCL_0419
45611    443888               CRIZOTINIB  ...     MDA-MB-468              CVCL_0419
45612    443894              CABAZITAXEL  ...     MDA-MB-468              CVCL_0419
45613    443897               VANDETANIB  ...     MDA-MB-468              CVCL_0419
45614    443945  TOPOTECAN HYDROCHLORIDE  ... 

In [ ]:
def target_protein_stith(x):
  print(x)
  url = "http://stitch.embl.de/api/tsv/interactors?identifier="+x+"&species=9606"
  res = pd.read_csv(url,delimiter=".")
  res['Drug_name'] = x
  res_02 = res.iloc[1:,]
  return (res_02)

def get_result_stitch(x):
  try:
    return (target_protein_stith(x))
  except:
    pass
 
res_target_protein_stitch = pd.concat(map(get_result_stitch,df_list_drug))
res_target_protein_stitch

In [35]:
res_target_protein_stitch.value_counts()

itemId           Drug_name   
ENSP00000464149  BLEOMYCIN       1
ENSP00000283916  METHOXSALEN     1
                 DACARBAZINE     1
                 ALLOPURINOL     1
ENSP00000283635  TEMOZOLOMIDE    1
                                ..
ENSP00000353375  VINORELBINE     1
ENSP00000352992  EXEMESTANE      1
ENSP00000352852  PD325901        1
ENSP00000352516  THIOGUANINE     1
ENSP00000001146  TRETINOIN       1
Length: 792, dtype: int64

### Remaining Drug

In [31]:
#filterremaining drug from dataset
remaining_drug = res_target_protein_stitch['Drug_name'].unique().tolist()
remain_df_01 = df[df["drug_row"].isin(remaining_drug)]
remain_df_02 = df[df["drug_col"].isin(remaining_drug)]
remain_df_03 = remain_df_02[['cell_line_name','drug_row','drug_col','synergy_loewe']]
remain_df_04 = remain_df_03.groupby(['cell_line_name','drug_row','drug_col']).agg({'synergy_loewe': 'mean'})
remain_df_04


synergy_loewe
cell_line_name drug_row                                           drug_col                   
BT-549         1-(5-DEOXYPENTOFURANOSYL)-5-FLUORO-4-{[(PENTYLO... ALLOPURINOL         -4.3700
                                                                  AMIFOSTINE          -3.2500
                                                                  ANASTROZOLE         -7.7600
                                                                  AZACYTIDINE        -34.8300
                                                                  BLEOMYCIN          -15.2300
...                                                                                       ...
ZR751          ZOLINZA                                            SN-38                0.3000
                                                                  SORAFENIB            1.8425
                                                                  SUNITINIB           -9.8175
                                                                  TEMOZOLOMIDE       -67.0150
                                                                  TOPOTECAN            3.4075

[22258 rows x 1 columns]

#Drug Structure (GCN layer)

In [ ]:
def tune_nn_model(hp):
    model = keras.Sequential()
    
    drop_rate = hp.Choice('drop_rate',
                            [
                              0.0,
                              0.1,
                              0.2,
                              0.3,
                              0.4,
                              0.5,
                              0.6,
                              0.7,
                              0.8,
                              0.9
                            ])

     
    model.add(keras.layers.Dense(
      units=28,
      activation="relu",
      input_shape=[df_x.shape[1]]
    ))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dropout(rate=drop_rate))
    model.add(keras.layers.Dense(1, activation='sigmoid')) 
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])

    model.compile(
        optimizer=keras.optimizers.SGD(lr, momentum=momentum),
        loss = 'binary_crossentropy',
        metrics = ['accuracy'])
    return model

In [ ]:
b_tuner = BayesianOptimization(
    tune_nn_model,
    objective='accuracy',
    max_trials=20,
    executions_per_trial=5,
    directory='test_dir_03',
    project_name='b_tune_nn'
)

INFO:tensorflow:Reloading Oracle from existing project test_dir_03/b_tune_nn/oracle.json


In [ ]:
df_x_x = df_x.to_numpy()
df_y_y = df_y.to_numpy()

In [ ]:
b_tuner.search(df_x_x,df_y_y)

In [ ]:
b_tuner.results_summary()

In [ ]:
# Retrieve the best model and display its architecture
best_model = b_tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 28)                112       
_________________________________________________________________
dropout (Dropout)            (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                464       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 593
Trainable params: 593
Non-trainable params: 0
_________________________________________________________________
